In [ ]:
"""
구간화 : 특정 변수를 범주형 변수로 변환
- 목적 : 강건한 모델 생성, 과적합 방지
- 수치형/범주형 변수에 모두 적용 가능
"""

In [6]:
import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt

<ipython-input-6-0c95b2fc7292>:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [4]:
"""
Load AirQualityUCI Data
"""

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv'

df = pd.read_csv(input_file,
                 index_col=[0],
                 parse_dates=[0],
                 date_parser=parser)

df.head()

,CO(GT),PT08.S1(CO),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),RH,AH,C6H6(GT)
Datetime,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,1046.0,166.0,1056.0,113.0,1692.0,1268.0,48.9,0.7578,11.9
2004-03-10 19:00:00,2.0,1292.0,955.0,103.0,1174.0,92.0,1559.0,972.0,47.7,0.7255,9.4
2004-03-10 20:00:00,2.2,1402.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,54.0,0.7502,9.0
2004-03-10 21:00:00,2.2,1376.0,948.0,172.0,1092.0,122.0,1584.0,1203.0,60.0,0.7867,9.2
2004-03-10 22:00:00,1.6,1272.0,836.0,131.0,1205.0,116.0,1490.0,1110.0,59.6,0.7888,6.5


In [7]:
# Visualization setup
%matplotlib
%config InlineBackend.figure_format = 'svg'

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
plt.ion() # enable the interactive mode

import seaborn as sns
sns.set()  # set plot styles

Using matplotlib backend: Qt5Agg


In [14]:
# Visualize the 'CO(GT)' column
co = df['CO(GT)'].copy()
co.interpolate(inplace=True) # inplace 원본에 덮어 쓰기를 한다 원본을 건드리기 싫어서 카피본에 덮어씀

In [16]:
"""
Binning
"""

# 처음과 끝을 구할 수 있다

max_val = co.max() # 관측된 값의 최댓값
min_val = co.min() # 관측된 값의 최솟값
print(max_val, min_val)

11.9 0.0


In [19]:
bins = np.linspace(min_val, max_val, 6) # 0부터 11.9사이의 값중에 6개간격의 값
bins

array([ 0.  ,  2.38,  4.76,  7.14,  9.52, 11.9 ])

In [23]:
# Labels for each bin
labels=['0 <=x<2.38', '2.38<=x<4.76', '4.76<=x<7.14',
       '7.14<=x<9.52', '9.52<=x<11.9']

In [25]:
# Convert the numerical values into the categorical values

# cut 메서드

df['bins'] = pd.cut(df['CO(GT)'], bins=bins,
                    labels=labels, include_lowest=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9357 entries, 2004-03-10 18:00:00 to 2005-04-04 14:00:00
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   CO(GT)         9357 non-null   float64 
 1   PT08.S1(CO)    8991 non-null   float64 
 2   PT08.S2(NMHC)  8991 non-null   float64 
 3   NOx(GT)        7718 non-null   float64 
 4   PT08.S3(NOx)   8991 non-null   float64 
 5   NO2(GT)        7715 non-null   float64 
 6   PT08.S4(NO2)   8991 non-null   float64 
 7   PT08.S5(O3)    8991 non-null   float64 
 8   RH             8991 non-null   float64 
 9   AH             8991 non-null   float64 
 10  C6H6(GT)       9357 non-null   float64 
 11  bins           9357 non-null   category
dtypes: category(1), float64(11)
memory usage: 886.6 KB


In [27]:
df['bins']

Datetime
2004-03-10 18:00:00    2.38<=x<4.76
2004-03-10 19:00:00      0 <=x<2.38
2004-03-10 20:00:00      0 <=x<2.38
2004-03-10 21:00:00      0 <=x<2.38
2004-03-10 22:00:00      0 <=x<2.38
                           ...     
2005-04-04 10:00:00    2.38<=x<4.76
2005-04-04 11:00:00    2.38<=x<4.76
2005-04-04 12:00:00    2.38<=x<4.76
2005-04-04 13:00:00      0 <=x<2.38
2005-04-04 14:00:00      0 <=x<2.38
Name: bins, Length: 9357, dtype: category
Categories (5, object): ['0 <=x<2.38' < '2.38<=x<4.76' < '4.76<=x<7.14' < '7.14<=x<9.52' < '9.52<=x<11.9']

In [31]:
# hist() 메서드 histogram 으로 보여준다

plt.hist(df['bins'], bins = 5)
plt.show()

In [34]:
# 로그 변환 : 우편향된 데이터 분포를 정규 분포에 가깝게 변환 
# - 이상치에 강건한 모델생성
# - 정규 분포에 적합한 알고리즘, 모수적 방법 적용이 용이해짐

"""
Log Transform
"""

# Distribution of original data
sns.distplot(df['PT08.S3(NOx)']) 

C:\Users\jaesu\anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:xlabel='PT08.S3(NOx)', ylabel='Density'>

In [36]:
# Calculate natural logarithm on 'CO(GT)' column

df['log'] = np.log10(df['PT08.S3(NOx)'])

In [38]:
df.min()

CO(GT)                  0.0
PT08.S1(CO)           647.0
PT08.S2(NMHC)         383.0
NOx(GT)                 2.0
PT08.S3(NOx)          322.0
NO2(GT)                 2.0
PT08.S4(NO2)          551.0
PT08.S5(O3)           221.0
RH                      9.2
AH                   0.1847
C6H6(GT)                0.0
bins             0 <=x<2.38
log                2.507856
dtype: object

In [40]:
# Distribution after log transform
# 분포를 확인하기 위한 시각화 정규분포에 가깝게 만들어짐(log를 사용한 후)

sns.distplot(df['log'])
plt.xlabel('log(PT08.S3(NoX))')
plt.show()

C:\Users\jaesu\anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [42]:
"""
One-hot Encoding
"""

# Make a dataset

emp_id = pd.Series([1, 2, 3, 4, 5])
gender = pd.Series(['Male', 'Female', 'Female', 'Male', 'Female'])
remarks = pd.Series(['Nice', 'Good', 'Great', 'Great', 'Nice'])

df_emp = pd.DataFrame()
df_emp['emp_id'] = emp_id
df_emp['gender'] = gender
df_emp['remarks'] = remarks

In [44]:
# Print DataFrame
df_emp

,emp_id,gender,remarks
0,1,Male,Nice
1,2,Female,Good
2,3,Female,Great
3,4,Male,Great
4,5,Female,Nice


In [46]:
# Print unique values for each column

print(df_emp['emp_id'].unique()) # 예측력을 가지는 변수가 아니다 
print(df_emp['gender'].unique())
print(df_emp['remarks'].unique())

[1 2 3 4 5]
['Male' 'Female']
['Nice' 'Good' 'Great']


In [48]:
# One-hot encoding the categorial values
# get_dummie 이진벡터로 나타내주는 메서드 

df_emp_encoded = pd.get_dummies(
    df_emp, columns = ['gender', 'remarks'])

df_emp_encoded

,emp_id,gender_Female,gender_Male,remarks_Good,remarks_Great,remarks_Nice
0,1,0,1,0,0,1
1,2,1,0,1,0,0
2,3,1,0,0,1,0
3,4,0,1,0,1,0
4,5,1,0,0,0,1


In [49]:
"""
Normalization

스케일링 : 모든 변수들이 비슷한 값 범위를 갖도록 변환하는 작업
- 정규화 모든 변수들을 0 ~ 1 사이의 값으로 스케일링 
- 표준화 : 표준편차를 기반으로 스케일링 수행 분포가 0을 중심으로하고 표쥰편차가 1이 되도록 스케일링 수행 
"""

# Visualize two columns of different scales
# 두 시계열 값에 차이가 많아서 CO값은 거의 보이지 않는다

plt.plot(df['CO(GT)'], label='CO') 
plt.plot(df['PT08.S2(NMHC)'], label='NMHC')
plt.legend(loc='best')

In [51]:
# Normalize the 'CO(GT)' column 
# 0 ~ 1 사이의 값

co = df['CO(GT)'].copy()
co_max = co.max()
co_min = co.min()

df['CO_Norm'] = (co - co_min) / (co_max - co_min)
df['CO_Norm']

Datetime
2004-03-10 18:00:00    0.218487
2004-03-10 19:00:00    0.168067
2004-03-10 20:00:00    0.184874
2004-03-10 21:00:00    0.184874
2004-03-10 22:00:00    0.134454
                         ...   
2005-04-04 10:00:00    0.260504
2005-04-04 11:00:00    0.201681
2005-04-04 12:00:00    0.201681
2005-04-04 13:00:00    0.176471
2005-04-04 14:00:00    0.184874
Name: CO_Norm, Length: 9357, dtype: float64

In [54]:
# Normalize the 'PT08.S2(NMHC)' column
# 0 ~ 1 사이의 값 

nmhc = df['PT08.S2(NMHC)'].copy()
nmhc_max = nmhc.max()
nmhc_min = nmhc.min()

df['NMHC_Norm'] = (nmhc - nmhc_min) / (nmhc_max - nmhc_min)
df['NMHC_Norm']

Datetime
2004-03-10 18:00:00    0.362097
2004-03-10 19:00:00    0.312398
2004-03-10 20:00:00    0.303659
2004-03-10 21:00:00    0.308575
2004-03-10 22:00:00    0.247406
                         ...   
2005-04-04 10:00:00    0.392135
2005-04-04 11:00:00    0.351720
2005-04-04 12:00:00    0.371382
2005-04-04 13:00:00    0.315674
2005-04-04 14:00:00    0.362643
Name: NMHC_Norm, Length: 9357, dtype: float64

In [58]:
# Visualized normalized columns
# 시각화

plt.plot(df['CO_Norm'], label='CO (normalized)')
plt.plot(df['NMHC_Norm'], label='NMHC (normalized)')
plt.legend(loc='best')

In [59]:
"""
Feature Split
split()를 이용한 영화 데이터의 특징 분할
"""

# Make untidy movie data 영화 데이터
movies = pd.Series(["The Godfather, 1972, Francis Ford Coppola",
                    "Contact, 1997, Robert Zemeckis",
                   "Parasite, 2019, Joon-ho Bong"])

movies

0    The Godfather, 1972, Francis Ford Coppola
1               Contact, 1997, Robert Zemeckis
2                 Parasite, 2019, Joon-ho Bong
dtype: object

In [60]:
# Divide movie data into title, year, director columns

lst_title = []
lst_year = []
lst_director = []

for val in movies:
    title, year, director = val.split(',')  # data split ,단위로 자른다
    lst_title.append(title)
    lst_year.append(year)
    lst_director.append(director)

print(lst_title)
print(lst_year)
print(lst_director)

['The Godfather', 'Contact', 'Parasite']
[' 1972', ' 1997', ' 2019']
[' Francis Ford Coppola', ' Robert Zemeckis', ' Joon-ho Bong']


In [62]:
# Make a DataFrame object
df_movie = pd.DataFrame()
df_movie['title'] = lst_title
df_movie['year'] = lst_year
df_movie['director'] = lst_director

df_movie

,title,year,director
0,The Godfather,1972,Francis Ford Coppola
1,Contact,1997,Robert Zemeckis
2,Parasite,2019,Joon-ho Bong
